In [17]:
import os
import pandas as pd
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI

In [18]:
os.environ["OPENAI_API_TYPE"] = "--"
os.environ["OPENAI_API_BASE"] = "--"
os.environ["OPENAI_API_KEY"] = "--"
os.environ["OPENAI_API_VERSION"] = "--"

### 데이터 로드

In [5]:
df = pd.read_csv("./data/poi_review.csv")

### 데이터 필터링

In [7]:
def get_categories(fullpath):
    categories = []
    splited = list(map(str.strip, fullpath.split(">")))
    for s in splited:
        s = s.strip()
        if s == "":
            continue

        categories_ = list(map(str.strip, s.split(",")))

        categories.extend(categories_)

    return categories

In [8]:
filtered_df = df[df['reviews'].apply(lambda x: isinstance(x, str) and str(x).strip() != '')]
filtered_df["scores"] = filtered_df["scores"].apply(eval)
filtered_df = filtered_df[filtered_df["scores"].apply(sum) / filtered_df["scores"].apply(len) >= 3.5]
filtered_df = filtered_df[filtered_df["reviews"].apply(len) > 50]
filtered_df["reviews"] = filtered_df["reviews"].apply(lambda x : x[1:-1])
filtered_df["categories"] = filtered_df["fullpath"].apply(get_categories)

In [9]:
filtered_df.head(5)

,poi_id,poi_name,visit_count,hcode,hname1,hname2,hname3,cate_id,fullpath,catename,...,landmarkyn,name1,name2,name3,name4,name5,time_list,scores,reviews,categories
5,9107967,서적백화점,108,1168067000,서울특별시,강남구,개포2동,212.0,"문화,예술>도서>서점",서점,...,N,"문화,예술",도서,서점,NaN,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[4, 5, 1, 5, 5, 5]",대치 학원가에서 사용하는 모든 교재는 여기에 다 있어요. 책 찾아달라고 말씀 드리면...,"[문화, 예술, 도서, 서점]"
7,14542282,알라딘중고서점,136,1111061500,서울특별시,종로구,종로1.2.3.4가동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[5, 4, 2, 5, 3, 4, 5, 5, 5, 5, 1, 4, 5, 1, 2, 1]",책 판매하러 갔는데 직원분들 엄청 친절하세요. 남/여 두분 다 너무 친절했어요. 주...,"[문화, 예술, 도서, 서점, 중고서점]"
8,434486987,알라딘중고서점,75,1130559500,서울특별시,강북구,번1동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[5, 5, 5, 5, 5, 5, 4, 1]",",전에 카페도 있었을 때가 더 좋았지만!,좋아요,친절하고 깔금하고 왠만한 책들은 다...","[문화, 예술, 도서, 서점, 중고서점]"
10,19511091,알라딘중고서점,46,1162064500,서울특별시,관악구,서원동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 4]","책 많고 직원분이 친절하십니다,매장이 크고 깨끗해서 좋아보였음.,알라딘 온라인으로 ...","[문화, 예술, 도서, 서점, 중고서점]"
11,1794507908,서울책보고,114,1171071000,서울특별시,송파구,잠실6동,880.0,"문화,예술>도서>서점>중고서점",중고서점,...,N,"문화,예술",도서,서점,중고서점,NaN,"[{""periodEtc"":null,""periodName"":""영업기간"",""timeLi...","[1, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 4, 5, 5, 3, 1]","직원들은 무례합니다,책이 다양하게 있진 않지만 조용하고 분위기가 좋아요 인스타그램에...","[문화, 예술, 도서, 서점, 중고서점]"


In [10]:
food_shop_df =  filtered_df[filtered_df["categories"].apply(lambda x : "음식점" in x)]

### 모델 선언

In [12]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

review_summarize_system_template = """
너는 "리뷰묶음"을 받아서 리뷰를 요약해주는 AI모델이야.
너는 입력으로 가게의 "리뷰묶음" 정보를 입력받아서 해당 리뷰를 요약해주는 문장을 생성해야 해.
이때 리뷰에서 부정적인 내용, 가게에 좋지 않은 내용, 기분이 나쁜 내용들은 절대로 리뷰 내용에 포함하지 말아야 해.
또한 "리뷰묶음"에 포함되어 있지 않았던 정보는 포함되면 안 돼.
그리고 추가적으로 리뷰 요약과 리뷰 내용을 통해 다음과 같은 내용 중 한 가지를 유츄해 선택해야 해
- "사람들이 주로 방문하는 시간대", "사람들이 주로 같이 방문하는 인원", "식당의 분위기", "식당 메뉴"
이때 주어진 리뷰를 기반으로 위의 정보들을 알 수 없는 경우에는 "알 수 없음"으로 응답해야 해
"""

review_summarize_question_example1 = """
[입력] :
- 가게 이름 : 달콤왕가탕후루
- 가게 위치 : 서울특별시 성동구 행당2동
- 카테고리 : [음식점, 간식]
- 리뷰 묶음 : 정말 맛있습니다. 또 방문하겠읍니다.,가성비 짱짱!!! 우리동네 맛집입니다^^매일 가고 싶네요 ^^,귤탕후루, 체리탕후루 중 귤탕후루가 맛있었당!,딸기탕후루 무슨 일입니까 진짜 ㅋㅋ이렇게 파실거면 가격을 내려야죠. 장난하십니까.딸기가 미니어쳐 수준  블루베리 크기만해요.너무나도 실망스럽습니다.
"""

review_summarize_answer_example1 = """
- 리뷰 요약 : 달콤왕가탕후루는 서울특별시 성동구에 위치하며 다양한 탕후루를 파는 가게이며, 동네에서 싸고 맛있는 가게로 소문이 나있습니다. 특히 귤탕후루가 맛있다고 합니다.
- 시간대 : 알 수 없음
- 인원 : 알 수 없음
- 분위기 : 알 수 없음
- 식당 메뉴 : 귤탕후루, 체리탕후루
"""

review_summarize_question_example2 = """
[입력] :
- 가게 이름 : 족발신선생
- 가게 위치 : 서울특별시 강서구 가양1동
- 카테고리 : [음식점, 한식, 육류, 고기, 족발, 보쌈, 족발신선생]
- 리뷰 묶음 : 다른 족발집 웨이팅하다가 너무 오래걸려서 여기로 왔어요.굉장히 만족스럽네요 ㅋㅋ 서비스라던가 메뉴 퀄리티 모두 좋습니다. 잘 먹고 가용 🫶,족발 정말 맛있어요.,여자직원들 싸가지없음,족발 맛있고,, 진짜 양 미텼고.. 서비스로 주는 어리굴젓 진짜 너무 맛있음 ㅠ 살코기 많은것 보다 껍질이 더 많은 느낌이라 쫄깃한 껍질을 더 좋아하면 가셔욤!! 사이드디쉬로 나온 짬뽕 순두부도 다 먹지 못할정도로 양이 많았어요~~강추~~,
"""

review_summarize_answer_example2 = """
- 리뷰 요약 : 족발신선생님 서울특별시 강서구 가양1동에 위치하는 한식 음식점이며 주로 족발과 보쌈을 판매하는 식당입니다. 다른 매장에 비해 웨이팅 없이 빠르게 입장할 수 있으며 서비스 및 퀄리티가 매우 좋습니다. 음식의 양 또한 많으며 특시 서비스로 주는 어리굴젓이 맛있다고 합니다.
- 시간대 : 알 수 없음
- 인원 : 알 수 없음
- 분위기 : 알 수 없음
- 식당 메뉴 : 족발, 짬뽕 순두부, 보쌈
"""

user_input = """
[입력] :
- 가게 이름 : {store_name}
- 가게 위치 : {store_location}
- 카테고리 : {category}
- 리뷰 묶음 : {reviews}
"""

In [13]:
sample = food_shop_df.loc[2916]
sample

poi_id                                                1298505567
poi_name                                                    섭생정식
visit_count                                                   42
hcode                                                 1132051500
hname1                                                     서울특별시
hname2                                                       도봉구
hname3                                                       창5동
cate_id                                                    124.0
fullpath                                                  음식점>한식
catename                                                      한식
catetype                                                       C
apicatename                                                  음식점
adultflag                                                      N
landmarkyn                                                     N
name1                                                        음식점
name2                    

In [20]:
chat = AzureChatOpenAI(azure_deployment="gpt-4")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=review_summarize_system_template),
        HumanMessage(content=review_summarize_question_example1),
        AIMessage(content=review_summarize_answer_example1),
        HumanMessage(content=review_summarize_question_example2),
        AIMessage(content=review_summarize_answer_example2),
        HumanMessage(content=user_input.format(store_name=sample["poi_name"], store_location=" ".join(
            [sample["hname1"], sample["hname2"], sample["hname3"]]), category=sample["categories"],
                                               reviews=sample["reviews"])),
    ]
)

chain = prompt | chat

In [21]:
chain.invoke({})

AIMessage(content='- 리뷰 요약 : 섭생정식은 서울특별시 도봉구에 있는 한식 음식점으로, 해산물이 신선하고 맛있다는 평가를 받고 있습니다. 특히 조미료를 사용하지 않고도 맛을 내는 것에 대해 많은 고객들이 만족감을 표현하고 있으며, 해산물 냄새가 나지 않아서 누구나 즐길 수 있다고 합니다. 또한, 주차는 인근 공영주차장을 이용하는 것이 좋다는 의견도 있습니다.\n- 시간대 : 알 수 없음\n- 인원 : 알 수 없음\n- 분위기 : 알 수 없음\n- 식당 메뉴 : 해산물, 산낙지 연포탕, 해물탕, 파전')